# Bert on Combined Corpus

In [1]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import string

2025-03-11 09:27:24.093543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741685244.228800  916734 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741685244.269804  916734 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 09:27:24.594275: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'transformers'

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print(f"Available GPUs: {gpus}")

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

2025-03-11 09:41:36.380479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 09:41:36.798411: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-11 09:41:36.798537: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-03-11 09:41:36.880577: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 09:41:38.446432: W tensorflow/stream_executor/platform/de

Available GPUs: []
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2025-03-11 09:41:40.105675: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-11 09:41:40.105911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-03-11 09:41:40.106006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-03-11 09:41:40.106096: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2025-03-11 09:41:40.200374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [17]:
df = pd.read_csv("../../datasets/Combined_Corpus/All.csv")

In [5]:
df.head()
print(df["Statement"].isnull().sum())

0


In [9]:
df

,Statement,Label,Statement_length,word_count,char_count,avg_word_length
0,More Than 100 Million Americans Are On Welfare...,0,4669,833,3836,4.605042
1,"There is no commitment to provide players, and...",1,2291,387,1904,4.919897
2,Does Uptick in Mysterious Booms Foretell Mega-...,0,8810,1450,7360,5.075862
4,"Andrew S. Grove, the longtime chief executive ...",1,12473,1980,10494,5.300000
5,"BLACKPOOL, England â€” The woman on the other ...",1,10886,1735,9152,5.274928
...,...,...,...,...,...,...
86526,Times reporters provided analysis and fact c...,1,10804,1808,8948,4.949115
86527,How did American intelligence officials come ...,1,4705,780,3897,4.996154
86528,WASHINGTON â€” The chiefs of Americaâ€™s in...,1,7898,1262,6612,5.239303
86529,Donald J. Trump said Wednesday that he would...,1,9199,1521,7650,5.029586


In [18]:
numar = df[df['word_count'] < 130].shape[0]
print("Numărul de rânduri cu word_count mai mic de 100:", numar)


Numărul de rânduri cu word_count mai mic de 100: 11985


In [19]:
print(df.shape)
df = df[df['word_count'] >= 30]
print(df.shape)

(86531, 6)
(82540, 6)


In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", '', text)
    text = re.sub(r"<.*?>+", '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)  # eliminare cuvinte cu cifre
    text = re.sub(r'\s+', ' ', text).strip()  
    text = re.sub(r'[“”‘’]', '', text)  
    return text

In [26]:
df['Statement'] = df['Statement'].apply(wordopt)

In [ ]:
texts = df["Statement"].tolist()
labels = df["Label"].tolist()

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [28]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_length = 128

def tokenize_texts(texts, tokenizer, max_length):
    return tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

train_tokens = tokenize_texts(train_texts, tokenizer, max_length)
test_tokens = tokenize_texts(test_texts, tokenizer, max_length)

In [ ]:
with strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
history = model.fit(
    x={
        "input_ids": train_tokens["input_ids"],
        "attention_mask": train_tokens["attention_mask"]
    },
    y=tf.convert_to_tensor(train_labels, dtype=tf.float32),
    validation_data=(
        {
            "input_ids": test_tokens["input_ids"],
            "attention_mask": test_tokens["attention_mask"]
        },
        tf.convert_to_tensor(test_labels, dtype=tf.float32)
    ),
    epochs=3,
    batch_size=16 * strategy.num_replicas_in_sync  
)

Epoch 1/3


2025-03-10 17:12:10.649791: W tensorflow/core/framework/dataset.cc:993] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


4127/4127 [==============================] - ETA: 0s - loss: 0.0719 - accuracy: 0.9707

2025-03-10 20:09:23.086532: W tensorflow/core/framework/dataset.cc:993] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2025-03-10 20:24:27.302604: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


4127/4127 [==============================] - 11537s 3s/step - loss: 0.0719 - accuracy: 0.9707 - val_loss: 0.0407 - val_accuracy: 0.9844
Epoch 2/3
 701/4127 [====>.........................] - ETA: 2:17:50 - loss: 0.0165 - accuracy: 0.9942

KeyboardInterrupt: 

In [ ]:
results = model.evaluate(
    x={
        "input_ids": test_tokens["input_ids"],
        "attention_mask": test_tokens["attention_mask"]
    },
    y=tf.convert_to_tensor(test_labels, dtype=tf.float32),
    batch_size=16
)
print(f"Test Accuracy: {results[1] * 100:.2f}%")

model.save_pretrained("../saved_models/saved_bert_model")
tokenizer.save_pretrained("../saved_models/saved_bert_tokenizer")